# Detecting temperature targets
##### author: Elizabeth A. Barnes and Noah Diffenbaugh
##### version: v0.1.0

```
conda create --name env-noah python=3.9
conda activate env-noah
pip install tensorflow==2.7.0
pip install tensorflow-probability==0.15.0
pip install --upgrade numpy scipy pandas statsmodels matplotlib seaborn palettable progressbar2 tabulate icecream flake8 keras-tuner sklearn jupyterlab black isort jupyterlab_code_formatter
pip install -U scikit-learn
pip install silence-tensorflow tqdm
conda install -c conda-forge cmocean cartopy
conda install -c conda-forge xarray dask netCDF4 bottleneck
conda install -c conda-forge nc-time-axis
```

Use the command
```python -m pip freeze > requirements.txt```
to make a pip installation list.

Data can be found here:
* https://www.earthsystemgrid.org/dataset/ucar.cgd.cesm2le.atm.proc.monthly_ave.TREFHT/file.html
* https://www.cesm.ucar.edu/projects/community-projects/LENS2/

In [ ]:
import xarray as xr
import numpy as np
# import pandas as pd
import matplotlib.pyplot as plt
# import cartopy.crs as ccrs

import os.path
from os import path
import subprocess


In [ ]:
dirName = '/Users/eabarnes/Desktop/big_data/orig_grid/'
dirReMapName = '/Users/eabarnes/Desktop/big_data/remap_grid/'
dirMergeName = '/Users/eabarnes/Desktop/big_data/merge_all/'
dirAnnualName = '/Users/eabarnes/Desktop/big_data/annual_mean/'

## Remap to coarser grid

In [ ]:
var = 'TREFHT'

for exp in ('BHISTsmbb','BSSP370smbb'):
    print('-----' + exp + '-----')
    for control_decade in (1231, 1251, 1281, 1301): 
        for member in np.arange(10,21):
            print('-----' + str(member) + '-----')

            #---------------------------------------------------------
            if(exp=='BHISTsmbb'):
                timetext = '185001-201412'
            elif(exp=='BSSP370smbb'):
                timetext = '201501-210012'
            else:
                raise ValueError('no such timetext')
            filename_merge = 'b.e21.' + str(exp) + '.f09_g17.LE2-' + str(control_decade) + '.0' + str(member) + '.cam.h0.' + str(var) + '.' + timetext + '.nc'
            filename_remap = filename_merge[:-3] + '.r180x90.nc'
            #---------------------------------------------------------

            if(os.path.isfile(dirReMapName+filename_remap)==True):
                print('remap file exists, continue')
            else:
                # grab variable only
                runText = "cdo -select,name=" + var + " " + dirName+filename_merge + " " + dirReMapName + "outfile.nc"
                print(runText)
                process = subprocess.Popen(runText.split(), stdout=subprocess.PIPE)
                output, error = process.communicate()       

                # remap to 2x2
                runText = 'cdo remapcon,r180x90 ' + dirReMapName + 'outfile.nc ' + dirReMapName+filename_remap 
                print(runText)
                process = subprocess.Popen(runText.split(), stdout=subprocess.PIPE)
                output, error = process.communicate()       

                # remove outfile.nc
                runText = 'rm ' + dirReMapName + 'outfile.nc'
                print(runText)
                process = subprocess.Popen(runText.split(), stdout=subprocess.PIPE)
                process = subprocess.Popen(runText.split(), stdout=subprocess.PIPE)
                output, error = process.communicate()  
                

## Merge historical and future

In [ ]:
var = 'TREFHT'

for control_decade in (1231, 1251, 1281, 1301): 
    for member in np.arange(10,21):
        print('-----' + str(member) + '-----')

        filename_hist = 'b.e21.BHISTsmbb.f09_g17.LE2-' + str(control_decade) + '.0' + str(member) + '.cam.h0.' + str(var) + '.' + '185001-201412' + '.r180x90.nc'
        filename_ssp  = 'b.e21.BSSP370smbb.f09_g17.LE2-' + str(control_decade) + '.0' + str(member) + '.cam.h0.' + str(var) + '.' + '201501-210012' + '.r180x90.nc'        
        filename_all  = 'b.e21.BHISTsmbb-BSSP370smbb.f09_g17.LE2-' + str(control_decade) + '.0' + str(member) + '.cam.h0.' + str(var) + '.' + '185001-210012' + '.r180x90.nc'
        # filename_short  = 'b.e21.BHISTsmbb-BSSP370smbb.f09_g17.LE2-' + str(control_decade) + '.0' + str(member) + '.cam.h0.' + str(var) + '.' + '195001-210012' + '.r180x90.nc'
        #---------------------------------------------------------

        if(os.path.isfile(dirMergeName+filename_all)==True):
            print('remap file exists, continue')
        else:
            # mergetime
            # runText = 'cdo mergetime ' + dirReMapName+filename_hist + ' ' + dirReMapName+filename_ssp + ' ' + dirMergeName+'timewrong.nc'
            runText = 'cdo mergetime ' + dirReMapName+filename_hist + ' ' + dirReMapName+filename_ssp + ' ' + dirMergeName+filename_all
            print(runText)
            process = subprocess.Popen(runText.split(), stdout=subprocess.PIPE)
            output, error = process.communicate()
            
        if(os.path.isfile(dirMergeName+filename_short)==True):
            print('remap file exists, continue')
        else:
            # mergetime
            runText = 'cdo selyear,1950/2101 ' + dirMergeName+filename_all + ' ' + dirMergeName+filename_short
            print(runText)
            process = subprocess.Popen(runText.split(), stdout=subprocess.PIPE)
            output, error = process.communicate()
            
#         if(os.path.isfile(dirMergeName+filename_short)==True):
#             print('remap file exists, continue')
#         else:
#             # mergetime
#             runText = 'cdo selyear,1950/2101 ' + dirMergeName+filename_all + ' ' + dirMergeName+filename_short
#             print(runText)
#             process = subprocess.Popen(runText.split(), stdout=subprocess.PIPE)
#             output, error = process.communicate()
                
            
            # shifttime
            # runText = 'cdo settaxis,1850-01-01,00:00:00,1month ' + dirMergeName+'timewrong.nc' + ' ' + dirMergeName+filename_all
            # print(runText)
            # process = subprocess.Popen(runText.split(), stdout=subprocess.PIPE)
            # output, error = process.communicate()
            
            # cdo settaxis,1920-01-15,00:00:00,1month tmp.nc filout.nc
            # remove timewrong.nc
            # runText = 'rm ' + dirMergeName + 'timewrong.nc'
            # print(runText)
            # process = subprocess.Popen(runText.split(), stdout=subprocess.PIPE)
            # output, error = process.communicate()  
            
            

## Take mean over all members

In [ ]:
runText = 'cdo ensmean ' + dirMergeName+'*185001-210012.r180x90.nc ' + dirMergeName + '/b.e21.BHISTsmbb-BSSP370smbb.f09_g17.LE2.cam.h0.TREFHT.185001-210012.r180x90.nc'
print(runText)
process = subprocess.Popen(runText.split(), stdout=subprocess.PIPE)
output, error = process.communicate()


In [ ]:
runText = 'cdo ensmean ' + dirMergeName+'*195001-210012.r180x90.nc ' + dirMergeName + '/b.e21.BHISTsmbb-BSSP370smbb.f09_g17.LE2.cam.h0.TREFHT.195001-210012.r180x90.nc'
print(runText)
process = subprocess.Popen(runText.split(), stdout=subprocess.PIPE)
output, error = process.communicate()


## Rename files and take annual mean

In [46]:
# https://ncar.github.io/esds/posts/2021/yearly-averages-xarray/

def weighted_temporal_mean(ds, var):
    """
    weight by days in each month
    """
    # Determine the month length
    month_length = ds.time.dt.days_in_month

    # Calculate the weights
    wgts = month_length.groupby("time.year") / month_length.groupby("time.year").sum()

    # Make sure the weights in each year add up to 1
    np.testing.assert_allclose(wgts.groupby("time.year").sum(xr.ALL_DIMS), 1.0)

    # Subset our dataset for our variable
    obs = ds[var]

    # Setup our masking for nan values
    cond = obs.isnull()
    ones = xr.where(cond, 0.0, 1.0)

    # Calculate the numerator
    obs_sum = (obs * wgts).resample(time="AS").sum(dim="time")

    # Calculate the denominator
    ones_out = (ones * wgts).resample(time="AS").sum(dim="time")

    # Return the weighted average
    return obs_sum / ones_out

In [52]:
import datetime
var = 'TREFHT'
X = np.zeros(30,251,90,180)
count = 0

for control_decade in (1231, 1251, 1281, 1301): 
    for member in np.arange(10,21):

        filename_all  = 'b.e21.BHISTsmbb-BSSP370smbb.f09_g17.LE2-' + str(control_decade) + '.0' + str(member) + '.cam.h0.' + str(var) + '.' + '185001-210012' + '.r180x90.nc'
        filename_ann  = filename_all[:-2] + 'annual.nc'
        
        if(os.path.isfile(dirMergeName + filename_all)==False):
            continue
        da = xr.open_dataset(dirMergeName + filename_all)
        #---------------------------------------------------------
        # fix the time stamp
        dates = da["time"]
        delta_time = datetime.timedelta(1)
        new_dates = dates - delta_time
        da["time"] = new_dates

        #---------------------------------------------------------
        # take the annual mean
         = weighted_temporal_mean(da,'TREFHT')
        da_annual.to_netcdf(dirAnnualName + filename_ann)

        #---------------------------------------------------------
        # concatenate all together
        X[count,:,:,:] = da_annual['TREFHT'].values()
        
# runText = 'cdo merge ' + dirAnnualName+'*.nc' + ' ' + dirAnnualName+'../b.e21.BHISTsmbb-BSSP370smbb.f09_g17.LE2.cam.h0.TREFHT.185001-210012.r180x90.annual.nc'
# print(runText)
# process = subprocess.Popen(runText.split(), stdout=subprocess.PIPE)
# output, error = process.communicate()
        
        

cdo merge /Users/eabarnes/Desktop/big_data/annual_mean/*.nc /Users/eabarnes/Desktop/big_data/annual_mean/../b.e21.BHISTsmbb-BSSP370smbb.f09_g17.LE2.cam.h0.TREFHT.185001-210012.r180x90.annual.nc


cdo    merge (Warning): Duplicate entry of parameter name __xarray_dataarray_variable__ in /Users/eabarnes/Desktop/big_data/annual_mean/b.e21.BHISTsmbb-BSSP370smbb.f09_g17.LE2-1231.012.cam.h0.TREFHT.185001-210012.r180x90.annual.nc!


In [58]:
da_annual.keys

<bound method Mapping.keys of <xarray.Dataset>
Dimensions:                        (time: 251, lon: 180, lat: 90)
Coordinates:
  * time                           (time) object 1850-01-01 00:00:00 ... 2100...
  * lon                            (lon) float64 0.0 2.0 4.0 ... 356.0 358.0
  * lat                            (lat) float64 -89.0 -87.0 -85.0 ... 87.0 89.0
Data variables:
    __xarray_dataarray_variable__  (time, lat, lon) float64 ...>

In [86]:
import datetime
var = 'TREFHT'
X = np.zeros((30,251,90,180))
count = 0

for control_decade in (1231, 1251, 1281, 1301): 
    for member in np.arange(10,21):
        filename_all  = 'b.e21.BHISTsmbb-BSSP370smbb.f09_g17.LE2-' + str(control_decade) + '.0' + str(member) + '.cam.h0.' + str(var) + '.' + '185001-210012' + '.r180x90.nc'
        filename_ann  = filename_all[:-2] + 'annual.nc'
        
        if(os.path.isfile(dirAnnualName + filename_ann)==False):
            continue
        da_annual = xr.open_dataset(dirAnnualName + filename_ann)
        print(filename_ann)
        #---------------------------------------------------------
        # concatenate all together
        X[count,:,:,:] = da_annual['__xarray_dataarray_variable__'].values
        count = count + 1
        
da = xr.DataArray(
    data=X,
    dims=["member","time","lat", "lon"],
    coords=dict(
        time=da_annual['time'].data,
        member=np.arange(0,30),
    ),
)
da.assign_coords(lon=da_annual['lon'])
da.assign_coords(lat=da_annual['lat'])
da.rename_vars({'__xarray_dataarray_variable__':'TREFHT'})

da.to_netcdf(dirAnnualName + 'b.e21.BHISTsmbb-BSSP370smbb.f09_g17.LE2.cam.h0.TREFHT.185001-210012.r180x90.annual.nc')

b.e21.BHISTsmbb-BSSP370smbb.f09_g17.LE2-1231.011.cam.h0.TREFHT.185001-210012.r180x90.annual.nc
b.e21.BHISTsmbb-BSSP370smbb.f09_g17.LE2-1231.012.cam.h0.TREFHT.185001-210012.r180x90.annual.nc
b.e21.BHISTsmbb-BSSP370smbb.f09_g17.LE2-1231.013.cam.h0.TREFHT.185001-210012.r180x90.annual.nc
b.e21.BHISTsmbb-BSSP370smbb.f09_g17.LE2-1231.014.cam.h0.TREFHT.185001-210012.r180x90.annual.nc
b.e21.BHISTsmbb-BSSP370smbb.f09_g17.LE2-1231.015.cam.h0.TREFHT.185001-210012.r180x90.annual.nc
b.e21.BHISTsmbb-BSSP370smbb.f09_g17.LE2-1231.016.cam.h0.TREFHT.185001-210012.r180x90.annual.nc
b.e21.BHISTsmbb-BSSP370smbb.f09_g17.LE2-1231.017.cam.h0.TREFHT.185001-210012.r180x90.annual.nc
b.e21.BHISTsmbb-BSSP370smbb.f09_g17.LE2-1231.018.cam.h0.TREFHT.185001-210012.r180x90.annual.nc
b.e21.BHISTsmbb-BSSP370smbb.f09_g17.LE2-1231.019.cam.h0.TREFHT.185001-210012.r180x90.annual.nc
b.e21.BHISTsmbb-BSSP370smbb.f09_g17.LE2-1231.020.cam.h0.TREFHT.185001-210012.r180x90.annual.nc
b.e21.BHISTsmbb-BSSP370smbb.f09_g17.LE2-1281.011.c